<h1>Interactive Philosophy Network Visualization</h1>

This file provides an interactive tool to visualize the relationships between over 500 philosophers who have published work before the 1950s.

<h3>Adjustable Parameters: </h3>

* **Philosopher:**
Select one philosopher to be the central focus of the visualization

* **Top References (1-20):** Number of philosophers most cited by the selected philosopher

* **Referenced By (1-20):** Number of philosophers who most frequently cite the selected philosopher

* **Line Threshold:**
Minimum citations required to display a connection between philosophers

* **Categories:**
Filter by philosophical topics/domains

<h3>Visualization elements:</h3>

**X-Axis:** Birth Year

**Y-Axis:** Total Outgoing References

**Bubble Size:** Total Incoming References

**Bubble Colors:**
* Blue: Selected philosopher
* Cyan: Top referenced by selected philosopher
* Pink: Top referencer of selected philosopher
* Purple: Both top referenced and referencer



# *Import Libraries:*

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import FancyArrowPatch
from ipywidgets import interact, Dropdown, IntSlider, SelectMultiple
import gdown

# *Download files*

In [2]:
file_id = "1XFmhtq1GR6b-suvka0LxmU9aEwplcVEl"  # From your Drive link
download_url = f"https://drive.google.com/uc?id={file_id}"
!wget -O references.csv {download_url}

file_id = "1quEEj3s9uhaYEbs0o_KnFenYl3JbB_pY"  # Replace with your actual file ID
gdown.download(f"https://drive.google.com/uc?id={file_id}", "helpers.py", quiet=False)

--2024-11-22 22:57:46--  https://drive.google.com/uc?id=1XFmhtq1GR6b-suvka0LxmU9aEwplcVEl
Resolving drive.google.com (drive.google.com)... 142.251.184.139, 142.251.184.100, 142.251.184.102, ...
Connecting to drive.google.com (drive.google.com)|142.251.184.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1XFmhtq1GR6b-suvka0LxmU9aEwplcVEl [following]
--2024-11-22 22:57:46--  https://drive.usercontent.google.com/download?id=1XFmhtq1GR6b-suvka0LxmU9aEwplcVEl
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.195.132, 2607:f8b0:4001:c11::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.195.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66328681 (63M) [application/octet-stream]
Saving to: ‘references.csv’

references.csv      100%[===================>]  63.26M   118MB/s    in 0.5s    

2024-11-22 22:57:50 (118 

Downloading...
From (original): https://drive.google.com/uc?id=1quEEj3s9uhaYEbs0o_KnFenYl3JbB_pY
From (redirected): https://drive.google.com/uc?id=1quEEj3s9uhaYEbs0o_KnFenYl3JbB_pY&confirm=t&uuid=a232325f-bd33-4297-ad90-9418ad3b1945
To: /content/helpers.py
100%|██████████| 6.03k/6.03k [00:00<00:00, 11.7MB/s]


'helpers.py'

# *Get Dataframe, Define Variables, Import Helper Functions:*

In [4]:
df = pd.read_csv('references.csv')
df['birth_year'] = df['birth_death'].str.extract(r'(-?\d{3,4})').astype(float)

figure_width = 7.5
figure_height = 4.5
font_size = 6
title_font_size = 10
point_size = 10
bubble_scale = 16
arrow_alpha = 0.6
arrow_width = 0.4

from helpers import (
    filter_dataframe,
    plot_scatter,
    configure_axes,
    create_widgets,
    row_matches_categories,
    get_display_philosophers,
    calculate_coordinates,
    get_philosopher_color,
    draw_connections,
)


# *Main Functions:*

In [5]:
# configure visualization
def plot_references(philosopher, categories_to_include=None, top_references=10, top_referenced_by=10, threshold=1):
    df_filtered = filter_dataframe(df, categories_to_include, philosopher)

    plt.close('all')
    fig, ax = plt.subplots(figsize=(figure_width, figure_height), dpi=150)
    display_philosophers, top_referenced, top_referenced_by_philosophers = get_display_philosophers(
        df_filtered, philosopher, top_references, top_referenced_by
    )

    df_display = df_filtered[
        (df_filtered['author_of_book'].isin(display_philosophers)) &
        (df_filtered['full_author_referenced'].isin(display_philosophers))
    ]
    restricted_referenced_counts = df_display['full_author_referenced'].value_counts()
    reference_counts = df_filtered['author_of_book'].value_counts()
    coordinates = calculate_coordinates(df_filtered, display_philosophers, reference_counts)

    plot_scatter(
        ax,
        coordinates,
        restricted_referenced_counts,
        philosopher,
        top_referenced,
        top_referenced_by_philosophers,
        philosopher,
        point_size,
        font_size,
        bubble_scale,
    )
    draw_connections(ax, df_display, coordinates, threshold, arrow_alpha, arrow_width)
    configure_axes(ax, philosopher, font_size, title_font_size)

    plt.tight_layout()
    fig_copy = fig
    plt.close()
    return fig_copy

# make interactive
def interactive_plot():
    sorted_philosophers = sorted(df['author_of_book'].unique())
    all_categories = df['predicted_category'].str.split(", ").explode().unique()
    category_options = ["All"] + list(all_categories)

    widgets = create_widgets(sorted_philosophers, category_options)
    interact(
        plot_references,
        philosopher=widgets[0],
        categories_to_include=widgets[1],
        top_references=widgets[2],
        top_referenced_by=widgets[3],
        threshold=widgets[4],
    )

# *Interactive Visualization:*

In [6]:
interactive_plot()

interactive(children=(Dropdown(description='Philosopher:', layout=Layout(width='500px'), options=('Aquinas', '…